<a href="https://colab.research.google.com/github/manishanaiyar/osha-safety-intelligence-suite/blob/main/BERT_NLP_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**NLP-Based Narrative Classification (BERT)**

**Objective:** Analyze raw incident narratives — actual text written by OSHA safety inspectors — and automatically classify them as High Risk (hospitalization) or Minor (outpatient/no treatment). Unlike structured models that read numbers and categories, BERT reads human language and understands context, making it uniquely suited for extracting risk signals from free-text incident descriptions.

**Why BERT Over Traditional NLP?**
Traditional NLP approaches (TF-IDF, bag-of-words) treat words as independent frequency counts. BERT is bidirectional — it reads every word in context of all surrounding words simultaneously:

```
Traditional: "worker" + "finger" + "caught" → separate signals
BERT:        "worker's finger was caught in running machinery"
             → understands the full contextual meaning as one unit

This matters for OSHA narratives where the same words mean
different things in different contexts:
"finger was amputated...treated and released" → Minor
"finger was amputated...hospitalized"         → High Risk
```

**Model Architecture:**
```
Input: Raw narrative text (up to 128 tokens)
         ↓
AutoTokenizer (bert-base-uncased)
Converts words → token IDs + attention mask
MAX_LEN = 128 (captures ~95% of OSHA narratives fully)
         ↓
12 Transformer layers (110M parameters total)
Each layer applies bidirectional self-attention
         ↓
[CLS] token — sentence-level representation
         ↓
Dropout layer (prevents overfitting)
         ↓
Linear classifier (768 → 2)
         ↓
Logits [Minor_score, HighRisk_score]
         ↓
Softmax → probability distribution
```

**Label Engineering:**
```
Hospitalized >= 1  →  Label 1 (High Risk)   17,280 cases (80.08%)
Hospitalized  = 0  →  Label 0 (Minor)        4,298 cases (19.92%)

IMPORTANT: Hospitalized measures administrative admission status,
not injury severity. This distinction became critical during
error analysis — see Key Finding below.
```

**Training Configuration:**
```
Base model:      bert-base-uncased (pre-trained on BookCorpus + Wikipedia)
Fine-tuning:     5 epochs on 17,262 OSHA narratives
Optimizer:       AdamW  lr=2e-5, eps=1e-8
                 (recommended rate for BERT fine-tuning —
                  large enough to learn, small enough not to
                  destroy pre-trained knowledge)
Loss function:   Weighted Cross Entropy
                 Minor weight:     ~4.02× (corrects 80/20 imbalance)
                 High Risk weight: ~1.00×
Scheduler:       Linear warmup (10% of steps) → linear decay
                 Prevents large early updates corrupting
                 pre-trained transformer weights
Batch size:      16
Hardware:        T4 GPU (CUDA)
Train/Val split: 80/20 stratified (random_state=42)
```

**Training Results — 5 Epochs:**
```
Epoch    Train Loss    Val Loss    Val Accuracy
─────────────────────────────────────────────
  1         —            —            —
  2         —            —            —
  3         —            —            —
  4         —            —            —
  5         —            —            —
(fill in from your training output)
```

**Validation Performance:**
```
=== Classification Report ===
               Precision  Recall   F1     Support
Minor (0)        0.75      0.85    0.80      860
High Risk (1)    0.96      0.93    0.94     3456
─────────────────────────────────────────────────
Accuracy                           0.91     4316
Macro avg        0.85      0.89    0.87     4316
Weighted avg     0.92      0.91    0.92     4316

=== Confusion Matrix ===
True Negatives  (correctly called minor):      734
True Positives  (correctly called high risk):  3207
False Positives (false alarms):                126
False Negatives (missed high risk ⚠️):         249

Missed high-risk cases: 249 of 3,456 (7.20%)
False alarm rate:        126 of   860 (14.65%)
```

**Real-Time Inference — Confirmed Working:**
```
Narrative: "a worker fell from scaffolding and sustained multiple
            fractures to the skull...transported to hospital"
→ High Risk (88.72%) ✅

Narrative: "an employee s right middle fingertip was amputated
            while pushing meat into an auger"
→ Minor (90.19%) ✅  (outpatient amputation — correct OSHA label)

Narrative: "an employee was operating a forklift when it tipped
            over...pinned underneath...hospitalized"
→ High Risk (89.37%) ✅

Narrative: "an employee s left index finger was amputated...
            treated and released from the emergency room"
→ Minor (77.55%) ✅  (treated and released = Minor in OSHA labeling)
```

**Key Finding — Ground Truth Label Analysis:**
During error analysis, we discovered that BERT learned the dataset correctly — but the label itself measures **hospital admission status**, not injury severity:

```
Amputation treated at outpatient facility  →  Hospitalized=0  →  Minor
Worker hospitalized for observation        →  Hospitalized=1  →  High Risk

Evidence from error analysis:
The 10 "most clearly Minor" cases (lowest HR probability) all
described amputations — correctly labeled Minor because the worker
was treated and released without hospital admission.

False positives included:
"worker was hospitalized and his left foot..." → labeled Minor
"resulting in an eye loss"                    → labeled Minor
Both were likely administrative data entry errors in OSHA records.
```

BERT correctly learned this administrative distinction. Our initial real-time inference tests failed because we used casual English test sentences (*"small scrape on his knee"*) that bear no resemblance to formal OSHA narrative language. Once tested with real OSHA-style narratives, the model performed correctly.

**Error Analysis — Where BERT Struggles:**
```
False Negatives (missed High Risk — 249 cases):
→ Out-of-distribution incidents: animal attacks, unusual
  machinery, night-shift repairs in rare industrial settings
→ BERT pattern-matches to common incident types and
  misclassifies rare events it hasn't seen enough of

False Positives (unnecessary alarms — 126 cases):
→ Incidents describing severe-sounding events that were
  administratively classified as outpatient
→ Some appear to be genuine OSHA data labeling errors
  (eye loss labeled Minor, hospitalization mentioned
  in a Minor-labeled narrative)

High confidence errors: 310 cases (>90% confidence wrong)
→ Model is certain but incorrect on ambiguous narratives
→ Reflects inherent noise in the Hospitalized label
```

**Model Saved:**
```
Location: /content/drive/MyDrive/OSHA_Project/BERT_OSHA_V2/
Files:
  config.json             model architecture + label mapping
  model.safetensors       trained weights (417.7 MB)
  tokenizer.json          vocabulary (30,522 tokens)
  tokenizer_config.json   tokenization settings

Label mapping (consistent throughout):
  0 = Minor     (Hospitalized = 0)
  1 = High Risk (Hospitalized >= 1)
```

**Conclusion:**
BERT achieved 91% accuracy and 93% recall on hospitalization prediction across 4,316 held-out validation samples. Error analysis revealed that the model learned the dataset's ground truth correctly — including the counterintuitive finding that amputation cases are labeled Minor when treated outpatient. This highlights a fundamental data quality consideration: the OSHA Hospitalized field is an administrative outcome measure, not a clinical severity measure. For production deployment, a composite severity label combining Hospitalized, Amputation, days away from work, and treatment type would produce a more meaningful and clinically accurate target variable.



In [ ]:
# ============================================================
# CELL 1: SETUP & DATA LOADING
# ============================================================
import pandas as pd
import numpy as np
import torch
import gc
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')

# Mount Drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/OSHA_Project/"

# Load frozen matrices
X_text    = pd.read_pickle(path + "X_text.pkl")
X_tabular = pd.read_pickle(path + "X_tabular.pkl")

# ============================================================
# Define Target Y
# Hospitalized >= 1 → High Risk (1)
# Hospitalized  = 0 → Minor     (0)
# This is our ground truth — consistent with all other notebooks
# ============================================================
Y_risk = (X_tabular['Hospitalized'].fillna(0) >= 1).astype(int)

# Free memory — X_tabular no longer needed
del X_tabular
gc.collect()

# Build NLP dataframe — remove rows with no narrative
df_nlp = pd.DataFrame({
    'Narrative':  X_text,
    'Risk_Label': Y_risk
})
df_nlp = df_nlp[
    df_nlp['Narrative'] != "No Narrative Provided"
].reset_index(drop=True)

print("✅ Data Loaded Successfully")
print(f"   NLP Matrix Shape: {df_nlp.shape}")
print(f"\n   Class Distribution:")
print(f"   High Risk (1): {(df_nlp['Risk_Label']==1).sum():,} "
      f"({(df_nlp['Risk_Label']==1).mean():.2%})")
print(f"   Minor     (0): {(df_nlp['Risk_Label']==0).sum():,} "
      f"({(df_nlp['Risk_Label']==0).mean():.2%})")

In [ ]:
# ============================================================
# CELL 2: TOKENIZATION
#
# WHY BERT needs tokenization:
# BERT doesn't read words — it reads token IDs.
# "fracture" → [19173]
# "conveyor"  → [24040]
#
# Every sentence gets:
# 1. Converted to token IDs (input_ids)
# 2. An attention mask — 1 = real token, 0 = padding
#    This tells BERT "pay attention here, ignore padding"
#
# MAX_LEN = 128:
# OSHA narratives are short. 128 tokens captures ~95% fully.
# Longer = more GPU memory, slower training, minimal gain.
# ============================================================
from transformers import AutoTokenizer

MAX_LEN = 128

print("Loading BERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print("Tokenizing all narratives...")
tokens = tokenizer(
    df_nlp['Narrative'].tolist(),
    max_length=MAX_LEN,
    padding='max_length',   # pad short sentences to exactly 128
    truncation=True,        # cut sentences longer than 128
    return_tensors='pt'     # return PyTorch tensors
)

X_input_ids      = tokens['input_ids']
X_attention_mask = tokens['attention_mask']
Y_labels         = torch.tensor(df_nlp['Risk_Label'].values)

print(f"\n✅ Tokenization Complete")
print(f"   Input IDs shape:      {X_input_ids.shape}")
print(f"   Attention mask shape: {X_attention_mask.shape}")
print(f"   Labels shape:         {Y_labels.shape}")
print(f"   Memory used:          ~{X_input_ids.element_size() * X_input_ids.nelement() / 1e6:.0f} MB")

In [ ]:
# ============================================================
# CELL 3: TRAIN/TEST SPLIT + DATALOADERS
#
# WHY stratify?
# Without it, the 80/20 split might accidentally put
# more Minor cases in train and more High Risk in test
# or vice versa. Stratify guarantees exact class ratio
# is preserved in both sets.
#
# WHY DataLoaders?
# GPU processes data in batches. DataLoader handles:
# - Batching (groups of 16 samples)
# - Shuffling (RandomSampler for train)
# - Sequential order (SequentialSampler for validation)
# ============================================================
from torch.utils.data import (TensorDataset, DataLoader,
                               RandomSampler, SequentialSampler)

BATCH_SIZE = 16

# Split — same random_state=42 for reproducibility
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    X_input_ids, Y_labels,
    test_size=0.2, random_state=42, stratify=Y_labels
)
train_masks, val_masks, _, _ = train_test_split(
    X_attention_mask, Y_labels,
    test_size=0.2, random_state=42, stratify=Y_labels
)

# ============================================================
# Class weights — tell loss function to penalize
# minority class (Minor) mistakes more heavily
# Formula: W_i = N_total / (n_classes × N_i)
# ============================================================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(Y_labels.numpy()),
    y=train_labels.numpy()
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float)

print("✅ Split Complete")
print(f"   Train size: {len(train_labels):,}  "
      f"({(train_labels==0).sum():,} minor, {(train_labels==1).sum():,} high risk)")
print(f"   Val size:   {len(val_labels):,}  "
      f"({(val_labels==0).sum():,} minor, {(val_labels==1).sum():,} high risk)")
print(f"\n   Class weights: Minor={weights_tensor[0]:.4f}, "
      f"High Risk={weights_tensor[1]:.4f}")

# Build DataLoaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset   = TensorDataset(val_inputs,   val_masks,   val_labels)

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=BATCH_SIZE
)
val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=BATCH_SIZE
)

print(f"\n   Training batches:   {len(train_dataloader)}")
print(f"   Validation batches: {len(val_dataloader)}")

In [ ]:
# ============================================================
# CELL 4: MODEL INITIALIZATION
#
# WHY BertForSequenceClassification?
# It's BERT with a classification head on top.
# Architecture:
#   Input tokens
#       ↓
#   12 Transformer layers (attention + feedforward)
#       ↓
#   [CLS] token representation (sentence summary)
#       ↓
#   Dropout (prevents overfitting)
#       ↓
#   Linear layer (768 → 2)
#       ↓
#   Logits [Minor_score, HighRisk_score]
#
# WHY AdamW optimizer?
# Adam with Weight Decay. Standard for BERT fine-tuning.
# lr=2e-5 is the recommended learning rate for BERT fine-tuning
# — large enough to learn, small enough not to destroy
# pre-trained weights.
#
# WHY linear scheduler with warmup?
# Learning rate starts at 0, ramps up for warmup_steps,
# then linearly decays to 0. Prevents large updates early
# in training that could corrupt pre-trained knowledge.
# ============================================================
from transformers import (BertForSequenceClassification,
                          get_linear_schedule_with_warmup)
from torch.optim import AdamW
import torch.nn as nn

# GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Hardware: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Load pre-trained BERT with classification head
# CONSISTENT label mapping throughout entire notebook:
# 0 = Minor, 1 = High Risk
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    id2label={0: "Minor", 1: "High Risk"},
    label2id={"Minor": 0, "High Risk": 1},
    output_attentions=False,
    output_hidden_states=False,
)
model.to(device)

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,    # recommended for BERT fine-tuning
    eps=1e-8    # numerical stability
)

# Training schedule
EPOCHS      = 5
total_steps = len(train_dataloader) * EPOCHS
warmup_steps = total_steps // 10  # 10% warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Weighted loss function
# Cross Entropy because we have 2 classes (not MSE)
# Weighted because Minor class is heavily underrepresented
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor.to(device))

print(f"\n✅ Model Initialized")
print(f"   Parameters:      {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable:       {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"   Epochs:          {EPOCHS}")
print(f"   Total steps:     {total_steps:,}")
print(f"   Warmup steps:    {warmup_steps:,}")
print(f"   Label mapping:   0=Minor, 1=High Risk (consistent)")

In [ ]:
# ============================================================
# CELL 5: TRAINING LOOP
#
# Each epoch:
# 1. Forward pass  → model makes predictions (logits)
# 2. Loss          → how wrong was it? (Cross Entropy)
# 3. Backward pass → calculate gradients (which direction to adjust)
# 4. Clip gradients → prevent exploding gradients (cap at 1.0)
# 5. Optimizer step → update weights in the right direction
# 6. Scheduler step → adjust learning rate
#
# model.train() vs model.eval():
# train() → enables dropout (adds noise, prevents overfitting)
# eval()  → disables dropout (deterministic predictions)
# ============================================================
import time

print("=" * 55)
print("BERT FINE-TUNING — 5 Epochs on OSHA Narratives")
print("=" * 55)

training_history = []

for epoch_i in range(EPOCHS):
    print(f"\n{'='*55}")
    print(f"Epoch {epoch_i + 1} / {EPOCHS}")
    print(f"{'='*55}")

    # ── TRAINING PHASE ──
    model.train()
    total_train_loss = 0
    t0 = time.time()

    for step, batch in enumerate(train_dataloader):
        b_input_ids  = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels     = batch[2].to(device)

        # Clear gradients from previous step
        model.zero_grad()

        # Forward pass → get raw scores (logits)
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        logits  = outputs.logits

        # Calculate weighted cross entropy loss
        loss = loss_fn(logits, b_labels)
        total_train_loss += loss.item()

        # Backward pass → compute gradients
        loss.backward()

        # Clip gradients → prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update weights and learning rate
        optimizer.step()
        scheduler.step()

        # Progress report every 200 batches
        if step % 200 == 0 and step != 0:
            elapsed = time.time() - t0
            print(f"  Batch {step:>4} / {len(train_dataloader)}  "
                  f"Loss: {loss.item():.4f}  "
                  f"Elapsed: {elapsed:.0f}s")

    avg_train_loss = total_train_loss / len(train_dataloader)

    # ── VALIDATION PHASE ──
    model.eval()
    total_val_loss = 0
    val_preds, val_true = [], []

    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)

        logits    = outputs.logits
        val_loss  = loss_fn(logits, b_labels)
        total_val_loss += val_loss.item()

        preds = torch.argmax(logits, dim=1).cpu().numpy()
        val_preds.extend(preds)
        val_true.extend(b_labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = np.mean(np.array(val_preds) == np.array(val_true))

    training_history.append({
        'epoch':      epoch_i + 1,
        'train_loss': avg_train_loss,
        'val_loss':   avg_val_loss,
        'val_acc':    val_accuracy
    })

    print(f"\n  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")
    print(f"  Val Acc:    {val_accuracy:.4f}")

print(f"\n{'='*55}")
print("✅ Training Complete!")
print(f"{'='*55}")

# Plot training curves
history_df = pd.DataFrame(training_history)
fig, axes  = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history_df['epoch'], history_df['train_loss'],
             'b-o', label='Train Loss')
axes[0].plot(history_df['epoch'], history_df['val_loss'],
             'r-o', label='Val Loss')
axes[0].set_title('Loss per Epoch')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].plot(history_df['epoch'], history_df['val_acc'],
             'g-o', label='Val Accuracy')
axes[1].set_title('Validation Accuracy per Epoch')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle('BERT Training History — 5 Epochs', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# CELL 6: FULL EVALUATION
# ============================================================
print("Running full evaluation on validation set...\n")

model.eval()
all_preds, all_true, all_probs = [], [], []

for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_input_mask)

    logits      = outputs.logits.detach().cpu()
    probs       = torch.nn.functional.softmax(logits, dim=-1).numpy()
    preds       = np.argmax(probs, axis=1)
    label_ids   = b_labels.cpu().numpy()

    all_preds.extend(preds)
    all_true.extend(label_ids)
    all_probs.extend(probs)

all_preds = np.array(all_preds)
all_true  = np.array(all_true)
all_probs = np.array(all_probs)

# Classification Report
print("=" * 55)
print("BERT Classification Report")
print("=" * 55)
print(classification_report(
    all_true, all_preds,
    target_names=['Minor (0)', 'High Risk (1)']
))

# Confusion Matrix
cm = confusion_matrix(all_true, all_preds)
tn, fp, fn, tp = cm.ravel()

fig, ax = plt.subplots(figsize=(6, 5))
ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=['Minor (0)', 'High Risk (1)']
).plot(ax=ax, colorbar=False, cmap='Blues')
ax.set_title("BERT — Confusion Matrix (Validation Set)", fontsize=12)
plt.tight_layout()
plt.show()

print(f"True Negatives  (correctly called minor):        {tn}")
print(f"True Positives  (correctly called high risk):    {tp}")
print(f"False Positives (false alarms):                  {fp}")
print(f"False Negatives (missed high risk ⚠️):           {fn}")
print(f"\nMissed high-risk: {fn} of {fn+tp} ({fn/(fn+tp):.2%})")
print(f"False alarms:     {fp} of {fp+tn} ({fp/(fp+tn):.2%})")

In [ ]:
# ============================================================
# CELL 7: REAL-TIME INFERENCE
#
# KEY LESSON FROM PREVIOUS NOTEBOOK:
# The predict function must use padding='max_length'
# AND pass attention_mask explicitly.
# This matches training tokenization exactly and
# prevents the model from collapsing to one class.
# ============================================================

def predict_incident(text):
    """
    Predict risk level from raw incident narrative text.

    Args:
        text: incident description string

    Returns:
        label:      'Minor' or 'High Risk'
        confidence: percentage 0-100

    Notes:
        - Uses padding='max_length' to match training tokenization
        - Uses model.config.id2label — never hardcoded mapping
        - Consistent: 0=Minor, 1=High Risk
    """
    model.eval()

    inputs = tokenizer(
        text,
        padding='max_length',    # must match training
        max_length=MAX_LEN,      # must match training (128)
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        )

    probs      = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred_idx   = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred_idx].item() * 100

    # Always use model's own config — never hardcode
    label = model.config.id2label[pred_idx]

    return label, confidence

# ============================================================
# Test suite — covers all severity levels
# ============================================================

test_cases = [
    # High Risk — hospitalized
    ("a worker fell from scaffolding and sustained multiple fractures "
     "to the skull and was transported to the hospital for emergency surgery.",
     "High Risk"),

    # Minor — amputation but outpatient
    ("an employee s right middle fingertip was amputated while "
     "pushing meat into an auger.",
     "Minor"),

    # High Risk — explicitly hospitalized
    ("an employee was operating a forklift when it tipped over. "
     "the employee was pinned underneath and hospitalized for "
     "injuries to the chest and ribs.",
     "High Risk"),

    # Minor — treated and released
    ("an employee s left index finger was amputated at the first "
     "digit while operating a punch press. the employee was treated "
     "and released from the emergency room.",
     "Minor"),

    # High Risk — chemical exposure + hospitalized
    ("a worker was exposed to sulfuric acid fumes and suffered "
     "severe chemical burns. the worker was hospitalized for treatment.",
     "High Risk"),

    # Minor — first aid only
    ("an employee caught his fingertip in a conveyor belt. "
     "first aid was administered on site. no further treatment needed.",
     "Minor"),

    # Ambiguous
    ("an employee was operating a punch press when his finger "
     "was caught. the extent of the injury was not immediately clear.",
     "Ambiguous"),
]



print("=" * 65)
print("BERT Real-Time Incident Risk Classifier")
print("Label mapping: 0=Minor, 1=High Risk (consistent)")
print("=" * 65)

correct = 0
total_known = 0

for i, (text, expected) in enumerate(test_cases, 1):
    label, conf = predict_incident(text)
    icon   = "🔴" if label == "High Risk" else "🟢"

    if expected != "Ambiguous":
        is_correct = (label == expected)
        status     = "✅" if is_correct else "❌"
        correct   += int(is_correct)
        total_known += 1
    else:
        status = "🤔"

    print(f"\nTest {i}: {icon} {label} ({conf:.2f}%) {status}")
    print(f"  Expected:  {expected}")
    print(f"  Narrative: {text[:80]}...")

print(f"\n{'='*65}")
print(f"Test accuracy: {correct}/{total_known} = {correct/total_known:.0%}")

In [ ]:
# ============================================================
# CELL 8: SHAP — Word-Level Explanations
#
# WHY SHAP?
# The model gives a prediction. SHAP explains WHY.
# It assigns each word a score:
#   Positive score → pushed toward High Risk
#   Negative score → pushed toward Minor
#
# FIX from previous notebook:
# Original SHAP function was missing attention_mask.
# BERT needs both input_ids AND attention_mask.
# Without attention_mask, BERT treats padding tokens as
# real words — completely distorting the explanation.
# ============================================================
!pip install shap -q
import shap

def predict_proba_for_shap(texts):
    """
    SHAP-compatible wrapper.
    Correctly passes attention_mask to BERT.
    Returns probability array shape (n_samples, 2).
    """
    inputs = tokenizer(
        list(texts),
        padding='max_length',
        max_length=MAX_LEN,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']  # ← critical fix
        )

    probs = torch.nn.functional.softmax(
        outputs.logits, dim=-1
    ).cpu().numpy()

    return probs

# Initialize explainer
print("Initializing SHAP explainer...")
explainer = shap.Explainer(predict_proba_for_shap, tokenizer)

# Explain two contrasting predictions
shap_texts = [
    "Worker's hand got stuck in a conveyor belt leading to severe fracture.",
    "An employee slipped on water in the breakroom and got a small scrape."
]

print("Computing SHAP values...\n")
shap_values = explainer(shap_texts)

print("=" * 55)
print("HIGH RISK — Words driving this prediction (red = High Risk)")
print("=" * 55)
shap.plots.text(shap_values[0])

print("\n" + "=" * 55)
print("MINOR — Words driving this prediction (blue = Minor)")
print("=" * 55)
shap.plots.text(shap_values[1])

In [ ]:
# ============================================================
# CELL 9: ERROR ANALYSIS
# Where does BERT struggle and why?
# ============================================================
print("Analyzing BERT's mistakes...\n")

val_df = pd.DataFrame({
    'Narrative': [
        tokenizer.decode(t, skip_special_tokens=True)
        for t in val_inputs
    ],
    'Actual':    all_true,
    'Predicted': all_preds,
    'Confidence': [max(p) * 100 for p in all_probs]
})

val_df['Actual_Label']    = val_df['Actual'].map(
    {0: 'Minor', 1: 'High Risk'})
val_df['Predicted_Label'] = val_df['Predicted'].map(
    {0: 'Minor', 1: 'High Risk'})
val_df['Correct']         = val_df['Actual'] == val_df['Predicted']

errors = val_df[~val_df['Correct']].copy()

print(f"Total errors:    {len(errors):,} of {len(val_df):,} "
      f"({len(errors)/len(val_df):.2%})")
print(f"False Positives: {fp} (called High Risk, was Minor)")
print(f"False Negatives: {fn} (called Minor, was High Risk ⚠️)")

# High confidence wrong predictions — most concerning
high_conf_errors = errors[errors['Confidence'] > 90].sort_values(
    'Confidence', ascending=False)
print(f"\nHigh confidence errors (>90%): {len(high_conf_errors)}")
print("These are cases where BERT was very wrong very confidently:\n")

print("--- False Negatives: Missed High Risk Cases ---")
fn_cases = errors[
    (errors['Actual'] == 1) &
    (errors['Predicted'] == 0)
].sort_values('Confidence', ascending=False).head(5)

for _, row in fn_cases.iterrows():
    print(f"\n  Narrative:  {row['Narrative'][:100]}...")
    print(f"  Actual:     {row['Actual_Label']}  "
          f"→  Predicted: {row['Predicted_Label']} "
          f"({row['Confidence']:.1f}% confident)")

print("\n--- False Positives: Unnecessary Alarms ---")
fp_cases = errors[
    (errors['Actual'] == 0) &
    (errors['Predicted'] == 1)
].sort_values('Confidence', ascending=False).head(5)

for _, row in fp_cases.iterrows():
    print(f"\n  Narrative:  {row['Narrative'][:100]}...")
    print(f"  Actual:     {row['Actual_Label']}  "
          f"→  Predicted: {row['Predicted_Label']} "
          f"({row['Confidence']:.1f}% confident)")

In [ ]:
# ============================================================
# CELL 10: SAVE MODEL TO DRIVE
# ============================================================
save_path = path + "BERT_OSHA_V2"

print(f"Saving model to: {save_path}")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"\n✅ Model saved successfully!")
print(f"   Location: {save_path}")
print(f"   Files saved:")
print(f"   - config.json          (model architecture)")
print(f"   - model.safetensors    (trained weights ~417MB)")
print(f"   - tokenizer.json       (vocabulary)")
print(f"   - tokenizer_config.json")

# Final summary
print(f"\n{'='*55}")
print("BERT OSHA V2 — Final Summary")
print(f"{'='*55}")
print(f"Model:          bert-base-uncased fine-tuned")
print(f"Task:           Incident narrative risk classification")
print(f"Training data:  {len(train_labels):,} OSHA narratives")
print(f"Epochs:         5")
print(f"Label mapping:  0=Minor, 1=High Risk (consistent)")
print(f"Validation acc: check classification report above")
print(f"Saved to:       {save_path}")